In [1]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("dev/train_features.csv")
train_metadata = pd.read_csv("dev/train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



MachineLearningAnalyses 
  variables: 20 rows, 21 columns
  metadata: 20 rows, 14 columns



In [2]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="MaxAbsScaler")
iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = "auto")

#scales and fits the model and upgrades analyses class to IsolationForestAnalyses
ana = scl.run(ana)
ana = iso.run(ana)
print(ana.__class__)

ana.plot_data()

<class 'src.StreamPort.machine_learning.analyses.IsolationForestAnalyses'>


In [3]:
ana.train()
fig_train_scores=ana.plot_scores()
fig_train_scores.write_image("dev/figures/fig_train_scores.png", width=1100, height= 350, scale = 3)
fig_train_scores.show()

In [4]:
test_data = pd.read_csv("dev/test_features.csv")
test_metadata = pd.read_csv("dev/test_metadata.csv")

ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)

fig_test_scores = ana.plot_scores()
fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()

    index  threshold     score  confidence    class
0     116  -0.065019 -0.104113        1.60  outlier
1     117  -0.065019 -0.037152        0.57   normal
2     118  -0.065019 -0.035823        0.55   normal
3     119  -0.065019 -0.047238        0.73   normal
4     120  -0.065019 -0.022850        0.35   normal
5     121  -0.065019 -0.024658        0.38   normal
6     122  -0.065019 -0.017810        0.27   normal
7     123  -0.065019 -0.025729        0.40   normal
8     124  -0.065019 -0.016917        0.26   normal
9     125  -0.065019 -0.016755        0.26   normal
10    126  -0.065019 -0.038445        0.59   normal
11    127  -0.065019 -0.021755        0.33   normal
12    128  -0.065019 -0.040595        0.62   normal
13    129  -0.065019 -0.037505        0.58   normal
14    130  -0.065019 -0.014818        0.23   normal
15    131  -0.065019 -0.025021        0.38   normal
16    132  -0.065019 -0.045749        0.70   normal


In [5]:
#ana.add_prediction()
#print(ana)

In [6]:
fig_test_features = ana.plot_data()
fig_test_features.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test_features.show()

In [7]:
test2_data = pd.read_csv("dev/test2_features.csv")
test2_metadata = pd.read_csv("dev/test2_metadata.csv")
test_samples = test2_metadata["index"]

ana.predict(test2_data, test2_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()

   index  threshold     score  confidence    class
0    266  -0.065019 -0.079234        1.22  outlier
1     47  -0.065019 -0.075752        1.17  outlier
2     17  -0.065019 -0.128332        1.97  outlier
3    119  -0.065019 -0.047238        0.73   normal
4     16  -0.065019 -0.113808        1.75  outlier


In [8]:
#ana.add_prediction()
#print(ana)

In [9]:
fig_test2_features = ana.plot_data()
fig_test2_features.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2_features.show()